In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re


import pickle
from etcpy import etc
from etcpy import tempDep
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
#df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    for react in mae.reactions:
        if match.match(react.id) and react.lower_bound != -1000:
            ori_lb = react.lower_bound
            react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae)}, r2_score(rexp,rae)

In [4]:
S = 0
SMin = 0
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux'])
for i in range(1,11):
    print(i)
    if i == 1:
        params = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Topt', 'dCpt'])
        rae, score = aerobic(params)

    else:
        params = tempDep.sample_data_uncertainty_with_constraint_small_increasing_topt(params, ['Topt', 'dCpt'])
        rae, score = aerobic(params)
    
    if score > S:
        S = score
        params.to_csv('../data/BestParamsTopt.csv')
        bestParams = params
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    elif score < SMin:
        SMin = score
        params.to_csv('../data/WorstParamsTopt.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirstTopt" + str(i) + ".pdf")
        plt.close(fig)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'grwoth', 'o2flux', 'gluFlux'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df = pd.concat([df, df1])

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-29
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.4595666883706956
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.6459457449137564
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.9015647074170651
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  1.0509554209029603
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  1.1752865057575623
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.280331803923219
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.3891787038630026
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4946446733079417
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4779903142628232
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3981688561995473
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.2861547303643923
NGAM is: 6.9000236503978725
Growth at  44.4534910975

In [5]:
df.to_csv('../results/modelResultsToptFirst.csv')

In [ ]:
S = 0
SMin = 0
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux'])
for i in range(1,101):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(bestParams, ['Tm'])
    rae, score = aerobic(newParams)

    if score > S:
        S = score
        newParams.to_csv('../data/BestParamsTm.csv')
        bestParamsTm = newParams
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirst" + str(i) + ".pdf")
        plt.close(fig)
    elif score < SMin:
        SMin = score
        newParams.to_csv('../data/WorstParams.csv')
    
        fig, ax1 = plt.subplots(figsize=(9, 6))
        ax2 = ax1.twinx()

        ax1.scatter(dfae_batch.index,dfae_batch['r'].values,label='Experimental Growth',color='k')
        ax1.plot(dfae_batch.index,rae['growth'],markersize=5,marker='o',label='Pred Growth')

        ax2.plot(dfae_batch.index,rae['glucose'],markersize=5,marker='o',label='Glucose Uptake', color='tab:brown')
        ax2.plot(dfae_batch.index,rae['oxygen'],markersize=5,marker='o',label='Oxygen Uptake', color='tab:gray')

        ax1.set_xlabel("Temperature ($^\circ$C)")
        ax1.set_ylabel("Aerobic $r_{max}$ (h$^{-1}$)")

        ax2.set_ylabel("Uptake fkux")

        fig.legend(fontsize=10, loc=2)
        fig.tight_layout()
        fig.savefig("../results/TmWigFirst" + str(i) + ".pdf")
        plt.close(fig)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'grwoth', 'o2flux', 'gluFlux'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df = pd.concat([df, df1])
if 'bestParams' not in globals():
    bestParams = newParams

1
NGAM is: 3.23


/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.05390115051279541
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.18188477203190195
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.35452289386932373
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5104981317395979
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6375913981150556
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.81806799658099
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.128300560545645
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.3617708806858302
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4761902595740386
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3536368704806856
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1668612811942687
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.09

/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


...

Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016


/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468


/home/aditi/.local/lib/python3.10/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  28.251819864188008 is:  0
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5050086043670625
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.8670816681863507
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1039901477713001
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3570125899886312
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4554415813242785
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4760314023819066
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3536352702386671
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1668596296497002
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.095963179155255
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:  0.6932918547944401
NGAM is: 7.226932905456837
Growth at  48.47717620988101 is:  0.22354552821614693
[0.27036    0.58194    0.636804   0.78318    0.4510794

In [ ]:
df.to_csv('../results/modelResultsToptFirstTmNext.csv')